In [1]:
!pip install tensorflow==2.4.1

     |████████████████████████████████| 394.3 MB 11 kB/s s eta 0:00:01   |█▋                              | 20.2 MB 4.3 MB/s eta 0:01:27     |███████████████████             | 234.0 MB 79.7 MB/s eta 0:00:03     |█████████████████████████████▍  | 362.3 MB 57.1 MB/s eta 0:00:01
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve, auc, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, TruncatedSVD

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [3]:
# Data loading
df_train = pd.read_csv("../input/widsdatathon2021/TrainingWiDS2021.csv")
df_test = pd.read_csv("../input/widsdatathon2021/UnlabeledWiDS2021.csv")
print(df_train.shape)
print(df_test.shape)

(130157, 181)
(10234, 180)


In [51]:
# Data type 
# ['hospital_id']
col_target = 'diabetes_mellitus'
cols_id = ['encounter_id', 'Unnamed: 0']
cols_cat = [i for i in df_train.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_train if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

In [52]:
df_ft_full = pd.concat( [df_train[cols_ft], df_test[cols_ft]], axis = 0)
df_ft_full[cols_num] = df_ft_full[cols_num].fillna(0)
df_ft_full[cols_cat] = df_ft_full[cols_cat].fillna('missing')
# df_ft_full = df_ft_full.reset_index(drop = True)
    
# df_ft_full[cols_cat] = df_ft_full[cols_cat].astype('object')
# df_ft_full[cols_num] = df_ft_full[cols_num].astype('float64')

In [53]:
df_train_full = pd.concat( [df_ft_full[:130157],df_train[[col_target]] ], axis = 1 )

In [55]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=128):
    dataframe = dataframe.copy()
    try:
        labels = dataframe.pop(col_target)
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    except:
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
                                                
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

df_to_train, df_to_val = train_test_split(df_train_full, test_size = 0.2)

In [61]:
batch_size = 128 # A small batch sized is used for demonstration purposes
ds_train = df_to_dataset(df_to_train, batch_size=batch_size)
ds_val = df_to_dataset(df_to_val, shuffle=False, batch_size=batch_size)
ds_test = df_to_dataset(df_ft_full[130157:], shuffle=False, batch_size=batch_size)

In [58]:
# FEATURE COLUMNS
fcs_num = [tf.feature_column.numeric_column(i) for i in cols_num]
fcs_cat = [
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            col, vocabulary_list = list(df_ft_full[col].drop_duplicates() )
        ), dimension = int(len(df_ft_full[col].drop_duplicates() ) * 0.9)
    )
    for col in cols_cat
]


In [59]:
# MLP Nets
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(fcs_num ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.015), 
              metrics = [tf.keras.metrics.AUC()])

In [60]:
history = model.fit( ds_train, validation_data = ds_val,
                   epochs = 10, batch_size = 128)

Epoch 1/10
814/814 [==============================] - 33s 28ms/step - loss: 0.5401 - auc_1: 0.7204 - val_loss: 0.4104 - val_auc_1: 0.8270
Epoch 2/10
814/814 [==============================] - 32s 33ms/step - loss: 0.4165 - auc_1: 0.8184 - val_loss: 0.4036 - val_auc_1: 0.8344
Epoch 3/10
814/814 [==============================] - 29s 30ms/step - loss: 0.4067 - auc_1: 0.8303 - val_loss: 0.4002 - val_auc_1: 0.8383
Epoch 4/10
814/814 [==============================] - 26s 27ms/step - loss: 0.3989 - auc_1: 0.8388 - val_loss: 0.3974 - val_auc_1: 0.8418
Epoch 5/10
814/814 [==============================] - 24s 24ms/step - loss: 0.3951 - auc_1: 0.8439 - val_loss: 0.3963 - val_auc_1: 0.8437
Epoch 6/10
814/814 [==============================] - 23s 23ms/step - loss: 0.3921 - auc_1: 0.8442 - val_loss: 0.3939 - val_auc_1: 0.8455
Epoch 7/10
814/814 [==============================] - 29s 30ms/step - loss: 0.3912 - auc_1: 0.8482 - val_loss: 0.3933 - val_auc_1: 0.8464
Epoch 8/10
814/814 [==============

In [62]:
df_test['diabetes_mellitus'] = model.predict(ds_test)

In [63]:
df_ss = df_test[['encounter_id', 'diabetes_mellitus']]
df_ss.to_csv('./submission.csv', index = False)